# Analyzing Topical Model of Facebook's Topics aiming towards Users. 

In [3]:
from __future__ import absolute_import, division, print_function
# for word vec
import gensim
from gensim.models import Word2Vec as w2v
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

import codecs
import glob
import multiprocessing
import os
import nltk
import pprint
import re
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

ImportError: No module named 'gensim'

In [ ]:
# split into words
def get_words(raw):
    """
    :raw: text that has not been cleaned yet
    output: Stemmed Tokens
    """
    from nltk.tokenize import word_tokenize
    import string
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    tokens = word_tokenize(raw)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    return stemmed

def get_topic_distribution(lda_model, raw_input, dictionary):
    """Return a vecor of topical distribution of each document or text. 
    :param lda_model: the output of the function gensim.models.ldamodel.LdaModel
    :param raw_imput: raw chinese policy text or doc
    :param dictionary: the output of corpora.Dictionary() function which is the vocab.
    """
    from pandas import DataFrame
    other_texts = [ # needs tokenized
        get_words(raw_input)
    ]
    #dictionary = Dictionary(sentences)
    other_corpus = [dictionary.doc2bow(text) for text in other_texts]
    unseen_doc = other_corpus[0]
    vector = lda_model[unseen_doc][0]
    return(DataFrame.from_records(vector)[1])

# Does politcal advertisements have an impact on the US November election? 
## Testing on random sample of Facebook users:
- Only English 
- From October to December
- Testing the US election in November vs FB ads political influence

In [ ]:
import random
df = pd.read_csv('/home/ubuntu/Desktop/dataset_eu19/user_a.csv')
random.seed(10)
RANDOM_SAMPLE = df.iloc[:, 0:22].sample(10000) # just taking a random sample of the population

In [ ]:
RANDOM_SAMPLE['publicationTime'] = pd.to_datetime(RANDOM_SAMPLE['publicationTime']).dt.date
RANDOM_SAMPLE = RANDOM_SAMPLE.sort_values(by='publicationTime')
RANDOM_SAMPLE.head()

In [ ]:
# break into tokens for each sentence
import time
start = time.time()
sent = RANDOM_SAMPLE.concatenatedText

sentences = []
for i in sent:
    sentences.append(get_words(i))
end = time.time()
print(end-start, 'seconds')

In [ ]:
import pyLDAvis.sklearn
import gensim
import os
import dill
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
from gensim.models import Word2Vec as w2v
import time

# Create Dictionary
start = time.time()
id2word = corpora.Dictionary(sentences)
dictionary = Dictionary(sentences)
# Create Corpus
texts = sentences

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           minimum_probability=0.0)



pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

#pyLDAvis.display(vis)

In [ ]:
pyLDAvis.show(vis)

In [ ]:
lda_model.print_topics()

## Merge the original dataset with the Topical Distribution spread 

In [ ]:
import time
start = time.time()
buckets = []
for i in RANDOM_SAMPLE.concatenatedText:
    buckets.append(get_topic_distribution(lda_model,i,dictionary))
groups= []
for i in range(0,len(buckets)):
    groups.append(pd.Series.to_frame((buckets[i])).T)

groups = pd.concat(groups).reset_index()
DATA =  pd.concat([RANDOM_SAMPLE.reset_index(drop=True), groups],axis=1).reset_index(drop=True)
end = time.time()
print(end-start)
import dill
dill.dump_session('FBenv.db')
#dill.load_session('FBenv.db')

In [ ]:
DATA.tail()

In [ ]:
DATA.columns = [           'ANGRY',             'HAHA',             'LIKE',
                   'LOVE',              'SAD',              'WOW',
          'displaySource',       'fblinktype',               'id',
           'images.count',  'impressionOrder',   'impressionTime',
                 'nature',        'permaLink',           'postId',
        'publicationTime',           'source',       'sourceLink',
               'timeline',             'user', 'concatenatedText',
         'concatLanguage',            'index',                  'Politics',
                        'Advertisement',                  'Other']

In [ ]:
DATA.tail()

In [ ]:
ORGANIC = DATA[DATA.nature == 'organic']
SPONSOR = DATA[DATA.nature == 'sponsored']

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pandas as pd
import math
import math
from bubbly.bubbly import add_slider_steps
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from plotly import tools
import plotly.figure_factory as ff 
import warnings
warnings.filterwarnings('ignore')
import os

#ORGANIC['publicationTime']=pd.to_datetime(ORGANIC['publicationTime']).dt.date
EX = pd.DataFrame(ORGANIC.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Advertisement',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Politics',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')


data = [trace1, trace2, trace3]

layout = go.Layout(
    
    legend=dict(x=0, y=-.13,
       orientation="h")
)



# Topical Distribution Spread from Organic Posts Before and After the Election

In [ ]:
fig = go.Figure(data=data, layout=layout)
iplot(fig) 

In [ ]:

EX = pd.DataFrame(SPONSOR.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Sales Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Political Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')


data = [trace1, trace2, trace3]

layout = go.Layout(
    
    legend=dict(x=0, y=-.13,
       orientation="h")
)



# Sponsored Posts Topical Spread Before and After the Election

In [ ]:
fig = go.Figure(data=data, layout=layout)
iplot(fig) 

# How much does each User's gets exposed to these Political Advertisements? 
- Study case: pick user with ID "churros-collards-crisp" because he or she has the most impressions

In [2]:
df.user.value_counts() # We will now ananlyze the User's exposure to these Politcal, Advertisements

NameError: name 'df' is not defined

In [ ]:
USER = df[df['user']=='churros-collards-crisp']
USER['publicationTime'] = pd.to_datetime(USER['publicationTime']).dt.date
USER = USER.sort_values(by='publicationTime')

sent = USER.concatenatedText
sentences = []
for i in sent:
    sentences.append(get_words(i))

id2word = corpora.Dictionary(sentences)
dictionary = Dictionary(sentences)
# Create Corpus
texts = sentences

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           minimum_probability=0.0)
lda_model.print_topics()

In [ ]:
buckets = []
for i in USER.concatenatedText:
    buckets.append(get_topic_distribution(lda_model,i,dictionary))
groups= []
for i in range(0,len(buckets)):
    groups.append(pd.Series.to_frame((buckets[i])).T)

groups = pd.concat(groups).reset_index()
USER_DATA =  pd.concat([USER.reset_index(drop=True), groups],axis=1).reset_index(drop=True)


In [ ]:
USER_DATA.columns = [           'ANGRY',             'HAHA',             'LIKE',
                   'LOVE',              'SAD',              'WOW',
          'displaySource',       'fblinktype',               'id',
           'images.count',  'impressionOrder',   'impressionTime',
                 'nature',        'permaLink',           'postId',
        'publicationTime',           'source',       'sourceLink',
               'timeline',             'user', 'concatenatedText',
         'concatLanguage',             'time',          'topic_0',
                'topic_1',          'topic_2',          'topic_3',
                'topic_4',          'topic_5',          'topic_6',
                'topic_7',          'topic_8',          'topic_9',
               'topic_10',         'topic_11',         'topic_12',
               'topic_13',         'topic_14',         'topic_15',
               'topic_16',         'topic_17',         'topic_18',
               'topic_19',         'topic_20',         'topic_21',
               'topic_22',         'topic_23',         'topic_24',
               'topic_25',         'topic_26',         'topic_27',
               'topic_28',         'topic_29',         'topic_30',
               'topic_31',         'topic_32',         'topic_33',
               'topic_34',         'topic_35',         'topic_36',
               'topic_37',         'topic_38',         'topic_39',
               'topic_40',         'topic_41',         'topic_42',
               'topic_43',         'topic_44',         'topic_45',
               'topic_46',         'topic_47',         'topic_48',
               'topic_49',            'index',                  'Advertisement',
                        'Other',                  'Politics']
USER_ORGANIC = USER_DATA[USER_DATA.nature == 'organic']
USER_SPONSOR = USER_DATA[USER_DATA.nature == 'sponsored']

In [ ]:
from bubbly.bubbly import add_slider_steps
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from plotly import tools
import plotly.figure_factory as ff 
import warnings
warnings.filterwarnings('ignore')
import os
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot


EX = pd.DataFrame(USER_ORGANIC.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Organic Forwarded Ads',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Organic Political Posts',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Organic Posts',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')

trace_median0 =  go.Scatter(x=['2018-11-07', '2018-11-07'],
                            y=[0,1],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=20,
                                       line=dict(width=1),
                                       color="black"
                                       ),
                            name="US Mid-term Election 2018",
                            visible=True
                            )

data_organic = [trace1, trace2, trace3, trace_median0]

layout = go.Layout(

    legend=dict(x=0, y=-.13,
       orientation="h")
)

In [ ]:
EX = pd.DataFrame(USER_SPONSOR.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Sponsored Advertisement',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Sponsored Politics',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Sponsors',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')

trace_median1 =  go.Scatter(x=['2018-11-07', '2018-11-07'],
                            #y=[0,1],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=20,
                                       line=dict(width=1),
                                       color="black"
                                       ),
                            name="US Mid-term Election 2018",
                            visible=True
                            )

data_sponsored = [trace1, trace2, trace3, trace_median1]

layout = go.Layout(
    yaxis=dict( autorange=True,
               fixedrange = False),
    legend=dict(x=0, y=-.13,
       orientation="h")
)

# How does Facebook impact A user's timeline impression during Political Elections?


## For USER ***churros-collards-crisp*** Personal timeline impression exposure (below)
> ## Organic Posts: political posts are about 10% higher in proportion before the midterm election on November 7th. 
> * We can conclude that he is more likely to be influenced by his own political social network. 

In [ ]:
fig = go.Figure(data=data_organic, layout=layout)
iplot(fig) 

> ## Sponsored Posts for User **churros-collards-crisp**: 
> * Growth of 2% to 19% just 3 days before the US midterm election. 
> * He may be heavily influenced by Sponsored Political ads 2 weeks before the election.

In [ ]:
fig = go.Figure(data=data_sponsored, layout=layout)
iplot(fig) 